In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np

#fonction pour transformer str en liste
import ast

In [2]:
chemin_bd = r"./bd_ignore/"

In [8]:
df_tmdb = pd.read_csv(chemin_bd + 'tmdb_full.csv')  # Dataset des films
df_names = pd.read_csv(chemin_bd + 'name.basics.tsv', sep='\t')  # Dataset des acteurs
df_title_principals = pd.read_csv(chemin_bd + 'title.principals.tsv', sep='\t')  # Dataset des acteurs

/var/folders/4f/28z_1f6d67b_wy1qrp_rvp6w0000gn/T/ipykernel_20170/1803814735.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv(chemin_bd + 'tmdb_full.csv')  # Dataset des films


In [23]:
#df_tmdb[['release_date', 'title']][df_tmdb['title']=='Too Much Johnson']

,release_date,title
57301,1919-12-01,Too Much Johnson
239557,2013-10-09,Too Much Johnson


In [26]:
df_tmdb.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,...,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,NaN,0,['Adventure'],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,...,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,...,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"['Drama', 'Comedy', 'Crime']",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,73,['fi'],Released,NaN,Ariel,False,7.046,248,['Villealfa Filmproductions'],['FI']
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0,"['Drama', 'Comedy', 'Romance']",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,76,['en'],Released,NaN,Shadows in Paradise,False,7.182,269,['Villealfa Filmproductions'],['FI']


In [4]:
df_title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N


In [5]:
df_names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [6]:
# prends que les acteurs du df de title principals

df_title_principals = df_title_principals[df_title_principals['category'].isin(['actor', 'actress'])]
df_title_principals['characters'] = df_title_principals['characters'].dropna()
df_title_principals= df_title_principals.dropna(subset=['characters'])
df_title_principals.reset_index(inplace=True, drop=True)

In [7]:
df_tp_tmdb = pd.merge(df_tmdb, df_title_principals, left_on='imdb_id', right_on='tconst', how='right')

In [8]:
df_tp_tmdb.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country', 'tconst', 'ordering', 'nconst',
       'category', 'job', 'characters'],
      dtype='object')

In [9]:
df_tp_tmdb.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_average,vote_count,production_companies_name,production_companies_country,tconst,ordering,nconst,category,job,characters
0,False,NaN,0.0,"['Drama', 'Documentary']",NaN,16624.0,tt0000005,xx,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,...,5.556,80.0,['Edison Studios'],['US'],tt0000005,1,nm0443482,actor,\N,"[""Blacksmith""]"
1,False,NaN,0.0,"['Drama', 'Documentary']",NaN,16624.0,tt0000005,xx,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,...,5.556,80.0,['Edison Studios'],['US'],tt0000005,2,nm0653042,actor,\N,"[""Assistant""]"
2,False,NaN,0.0,['Documentary'],NaN,159895.0,tt0000007,xx,Corbett and Courtney Before the Kinetograph,James J. Corbett and Peter Courtney meet in a ...,...,4.984,32.0,['Edison Studios'],NaN,tt0000007,1,nm0179163,actor,\N,\N
3,False,NaN,0.0,['Documentary'],NaN,159895.0,tt0000007,xx,Corbett and Courtney Before the Kinetograph,James J. Corbett and Peter Courtney meet in a ...,...,4.984,32.0,['Edison Studios'],NaN,tt0000007,2,nm0183947,actor,\N,\N
4,False,NaN,0.0,['Documentary'],NaN,105158.0,tt0000008,xx,Edison Kinetoscopic Record of a Sneeze,A man (Thomas Edison's assistant) takes a pinc...,...,4.859,71.0,['Edison Studios'],NaN,tt0000008,1,nm0653028,actor,\N,"[""Sneezing Man""]"


In [10]:
# prend que le nom des acteurs et la colonne nconst qui servira à faire le merge

df_names2 = df_names.copy()
df_names2 = df_names2[['nconst', 'primaryName']]
df_names2.head(1)

,nconst,primaryName
0,nm0000001,Fred Astaire


In [11]:
tp_tmdb_n = pd.merge(df_tp_tmdb, df_names2, left_on='nconst', right_on='nconst', how='left')

In [12]:
tp_tmdb_n.head(15)

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_count,production_companies_name,production_companies_country,tconst,ordering,nconst,category,job,characters,primaryName
0,False,NaN,0.0,"['Drama', 'Documentary']",NaN,16624.0,tt0000005,xx,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,...,80.0,['Edison Studios'],['US'],tt0000005,1,nm0443482,actor,\N,"[""Blacksmith""]",Charles Kayser
1,False,NaN,0.0,"['Drama', 'Documentary']",NaN,16624.0,tt0000005,xx,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,...,80.0,['Edison Studios'],['US'],tt0000005,2,nm0653042,actor,\N,"[""Assistant""]",John Ott
2,False,NaN,0.0,['Documentary'],NaN,159895.0,tt0000007,xx,Corbett and Courtney Before the Kinetograph,James J. Corbett and Peter Courtney meet in a ...,...,32.0,['Edison Studios'],NaN,tt0000007,1,nm0179163,actor,\N,\N,James J. Corbett
3,False,NaN,0.0,['Documentary'],NaN,159895.0,tt0000007,xx,Corbett and Courtney Before the Kinetograph,James J. Corbett and Peter Courtney meet in a ...,...,32.0,['Edison Studios'],NaN,tt0000007,2,nm0183947,actor,\N,\N,Peter Courtney
4,False,NaN,0.0,['Documentary'],NaN,105158.0,tt0000008,xx,Edison Kinetoscopic Record of a Sneeze,A man (Thomas Edison's assistant) takes a pinc...,...,71.0,['Edison Studios'],NaN,tt0000008,1,nm0653028,actor,\N,"[""Sneezing Man""]",Fred Ott
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0000009,1,nm0063086,actress,\N,"[""Miss Geraldine Holbrook (Miss Jerry)""]",Blanche Bayliss
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0000009,2,nm0183823,actor,\N,"[""Mr. Hamilton""]",William Courtenay
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,tt0000009,3,nm1309758,actor,\N,"[""Chauncey Depew - the Director of the New Yor...",Chauncey Depew
8,False,NaN,0.0,['Documentary'],NaN,194085.0,tt0000011,de,Akrobatisches Potpourri,Eight circus performers known as the Grunato f...,...,15.0,[],NaN,tt0000011,1,nm3692297,actor,\N,"[""Acrobats""]",Grunato
9,False,/A0hONP6cxTthAsw6IvPMX4Aowkr.jpg,0.0,['Comedy'],NaN,82120.0,tt0000014,fr,Arroseur et arrosé,"A gardener is watering his flowers, when a mis...",...,214.0,"['Lumière', 'Société des Etablissements L. Gau...",NaN,tt0000014,1,nm0166380,actor,\N,"[""The Gardener""]",François Clerc


In [13]:
tp_tmdb_n.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companies_name',
       'production_companies_country', 'tconst', 'ordering', 'nconst',
       'category', 'job', 'characters', 'primaryName'],
      dtype='object')

In [14]:
tp_tmdb_n.to_csv(chemin_bd + 'resultat/tp_tmdb_n.csv', index = False)

/var/folders/4f/28z_1f6d67b_wy1qrp_rvp6w0000gn/T/ipykernel_19654/889638457.py:2: DtypeWarning: Columns (0,1,3,4,6,7,8,9,11,12,13,16,17,18,19,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tp_tmdb_n2 = pd.read_csv(chemin_bd + 'resultat/tp_tmdb_n.csv')


In [52]:
tp_tmdb_n2 = pd.read_csv(chemin_bd + 'resultat/tp_tmdb_n.csv')

/var/folders/4f/28z_1f6d67b_wy1qrp_rvp6w0000gn/T/ipykernel_20170/2004208016.py:1: DtypeWarning: Columns (0,1,3,4,6,7,8,9,11,12,13,16,17,18,19,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tp_tmdb_n2 = pd.read_csv(chemin_bd + 'resultat/tp_tmdb_n.csv')


In [53]:
tp_tmdb_n3 = tp_tmdb_n2.copy()

In [54]:
# Nettoyer et convertir la colonne 'release_date' en datetime dans df_tmdb
tp_tmdb_n3['release_date'] = pd.to_datetime(tp_tmdb_n3['release_date'], errors='coerce')
tp_tmdb_n3['year'] = tp_tmdb_n3['release_date'].dt.year

# Supprimer les lignes sans 'year' ou 'genres'
tp_tmdb_n3 = tp_tmdb_n3.dropna(subset=['year', 'genres'])

# Filtrer les films des années 2000
tp_tmdb_n3 = tp_tmdb_n3[(tp_tmdb_n3['year'] >= 2000) & (tp_tmdb_n3['year'] <= 2024)]

# filtrer les genres
tp_tmdb_n3 = tp_tmdb_n3[tp_tmdb_n3['genres'].str.contains("|".join(['Action', 'Drama', 'Comedy', 'Horror', 'Family', 'Adventure', 'Animation', 'Thriller']))]

# Exploser les genres associés (genres)
tp_tmdb_n3 = tp_tmdb_n3.assign(genres=tp_tmdb_n3['genres'].str.split(','))
tp_tmdb_n3 = tp_tmdb_n3.explode('genres')  # Une ligne par genre associé

# # Merge avec le dataset des acteurs sur 'imdb_id'
# df_tmdb['imdb_id'] = df_tmdb['imdb_id'].str.strip()  # Nettoyer d'éventuels espaces

# Enlever les []
tp_tmdb_n3['genres']=tp_tmdb_n3['genres'].apply(lambda x : str(x).replace('[','').replace(']',''))
tp_tmdb_n3['genres']=tp_tmdb_n3['genres'].apply(lambda x : str(x).replace("'",'').replace("'",''))

# df_tmdb2 = df_tmdb.copy()
# df_merged = pd.merge(df_tmdb, df_names, left_on='imdb_id', right_on='knownForTitles', how='left')


In [55]:
# Ajouter les noms des acteurs groupés par film
tp_tmdb_n3['actors'] = tp_tmdb_n3.groupby('imdb_id')['primaryName'].transform(lambda x: ', '.join(x.dropna()))
tp_tmdb_n3 = tp_tmdb_n3.drop_duplicates(subset=['imdb_id'])  # Supprimer les doublons

# Sélection des colonnes pertinentes pour le modèle
features = ['popularity', 'vote_average', 'vote_count', 'budget', 'revenue', 'runtime']
df_filtered_new = tp_tmdb_n3[features + ['genres', 'actors', 'title', 'imdb_id', 'year']].fillna(0)

In [ ]:
#list(df_filtered_new['year'].unique())

In [24]:
#df_filtered_new['actors']=df_filtered_new['actors'].apply(lambda x: set(x))

In [56]:
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: set(x.split(',')))

In [57]:
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).replace('{','').replace('}',''))
df_filtered_new['actors'] = df_filtered_new['actors'].apply(lambda x: str(x).replace("'",'').replace("'",''))

In [58]:
df_filtered_new.head()

,popularity,vote_average,vote_count,budget,revenue,runtime,genres,actors,title,imdb_id,year
216737,4.131,5.500,20.0,0.0,0.0,66.0,Comedy,"Arlene Francis, Ruth Ford, Orson Welles, E...",Too Much Johnson,tt0030881,2013.0
256780,15.770,6.326,1187.0,48000000.0,76019048.0,118.0,Romance,"Paxton Whitehead, Natasha Lyonne, Meg Ryan,...",Kate & Leopold,tt0035423,2001.0
386477,1.400,5.100,11.0,0.0,0.0,22.0,Horror,"Duke Moore, Dudley Manlove, Jeannie Stevens",Final Curtain,tt0050396,2012.0
492521,0.886,5.500,4.0,0.0,0.0,63.0,Drama,"Néstor Cantillana, Shenda Román, Luis Vilch...",The Tango of the Widower and Its Distorting Mi...,tt0062336,2020.0
506352,3.645,9.470,33.0,0.0,0.0,90.0,Animation,"Vic Perrin, Nicole Jaffe, Jean Vander Pyl, ...",Scooby-Doo's Creepiest Capers,tt0063950,2000.0


In [59]:
# on les met dans un fichier plus court pour la recherche
df_filtered_new.to_csv(chemin_bd + 'resultat/df_filtered_new.csv', index = False)

In [60]:
df_filtered_newactor = pd.read_csv(chemin_bd + 'resultat/df_filtered_new.csv')

In [61]:
# on selection une colonne interemediare avec deux acteurs
df_filtered_newactor['first_two_actors'] = df_filtered_newactor['actors'].apply(
    lambda x: ','.join(x.split(',')[:3]) if isinstance(x, str) else None
)

# on met en liste dans une nouvelle colonne 
df_filtered_newactor['two_actors'] = df_filtered_newactor['first_two_actors'].apply(lambda x: x.split(","))
df_filtered_newactor['two_actors'] = df_filtered_newactor['two_actors'].apply(lambda x: set(x))

# on supprime les colonnes inutiles
df_filtered_newactor = df_filtered_newactor.drop(columns =['first_two_actors'])  #'actors', 
df_filtered_newactor.head()

,popularity,vote_average,vote_count,budget,revenue,runtime,genres,actors,title,imdb_id,year,two_actors
0,4.131,5.500,20.0,0.0,0.0,66.0,Comedy,"Arlene Francis, Ruth Ford, Orson Welles, E...",Too Much Johnson,tt0030881,2013.0,"{ Orson Welles, Arlene Francis, Ruth Ford}"
1,15.770,6.326,1187.0,48000000.0,76019048.0,118.0,Romance,"Paxton Whitehead, Natasha Lyonne, Meg Ryan,...",Kate & Leopold,tt0035423,2001.0,"{ Paxton Whitehead, Meg Ryan, Natasha Lyonne}"
2,1.400,5.100,11.0,0.0,0.0,22.0,Horror,"Duke Moore, Dudley Manlove, Jeannie Stevens",Final Curtain,tt0050396,2012.0,"{ Dudley Manlove, Duke Moore, Jeannie Stevens}"
3,0.886,5.500,4.0,0.0,0.0,63.0,Drama,"Néstor Cantillana, Shenda Román, Luis Vilch...",The Tango of the Widower and Its Distorting Mi...,tt0062336,2020.0,"{ Néstor Cantillana, Shenda Román, Luis Vi..."
4,3.645,9.470,33.0,0.0,0.0,90.0,Animation,"Vic Perrin, Nicole Jaffe, Jean Vander Pyl, ...",Scooby-Doo's Creepiest Capers,tt0063950,2000.0,"{ Nicole Jaffe, Vic Perrin, Jean Vander Pyl}"


In [62]:
df_filtered_newactor.to_csv(chemin_bd + 'resultat/df_filtered_newactor.csv')

In [63]:
df_filtered_nv = pd.read_csv(chemin_bd + 'resultat/df_filtered_newactor.csv')